ALIEXPRESS WEBSCRAPER

In [117]:
#driver = webdriver.Chrome() 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 8.80MB/s]


-----FUNCTION DEFINITION & MODULES IMPORT-----

In [2]:
# IMPORTING MODULES
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import random 
import numpy as np


In [3]:
#THIS FUNCTION APPLY A TIME.SLEEP WITH A RANDOM VALUE WHICH IS CHOSEN BETWEEN 2 INPUTS:
#INPUT1=T1  -> MINIMUM TIME SLEEP
#INPUT2=T12 -> MAXIMUM TIME SLEEP
def time_randomizer (t1,t2):
    ran_time=random.uniform(t1,t2)
    time.sleep(ran_time)

In [4]:
def selenium_init(url):
    driver = webdriver.Chrome()  # Activate driver for Chrome in this case
    time_randomizer(1,3)
    # Open Url in the browser
    driver.get(url)
    time_randomizer(2,4)
    return driver

In [5]:
def enter_aliexpress():
    url='https://es.aliexpress.com/'
    driver=selenium_init(url)
    title = driver.title
    print('Webpage Title:', title)
    if len(title)>0:
        print('Successful connection \n')
    else:
        print('Conection failed \n')
    return driver

In [6]:
def starting_setup ():
    
    #ENTERING ALIEXPRESS
    driver = enter_aliexpress() 
    html=driver.page_source
    soup = BeautifulSoup(html)

    #ACCEPTING COOKIES
    time_randomizer(2,7)
    cookies=driver.find_element(By.CLASS_NAME, "btn-accept")
    cookies.click()
    driver.maximize_window() 

    #GOING TO VIDEOJUEGOS
    time_randomizer(3,7)
    driver.get("https://www.aliexpress.com/category/44/consumer-electronics.html")
    time_randomizer(2,5)
    videojuegos=driver.find_elements(By.XPATH, "//*[contains(text(), 'Videojuegos')]")[1]
    videojuegos.click()

    #SORTING BY NUMBER OF ORDERS
    driver.get("https://es.aliexpress.com/category/204003301/video-games.html")
    time_randomizer(3,6)
    sort_by_orders=driver.find_elements(By.XPATH, "//*[contains(text(),'Pedidos')]")[2]
    sort_by_orders.click()

    #SORTING BY RATING --> RATING>=4
    driver.get("https://es.aliexpress.com/category/204003301/video-games.html?CatId=204003301&g=y&isCategoryBrowse=true&isrefine=y&sortType=total_tranpro_desc&trafficChannel=main")
    time_randomizer(2,5)
    sort_by_rating=driver.find_elements(By.CLASS_NAME, "comet-checkbox-icon")[3]
    sort_by_rating.click()

    #SCROLLING TILL THE BOTTOM OF THE WEBPAGE
    for y in range (1,13):
        height = y*1080
        driver.execute_script(f"window.scrollTo(0, {height})") 
        time_randomizer(1,2)

    return driver


In [10]:
#THIS FUNCTION SCRAP THE DATA FROM THE CURRENT WEBPAGE & RETURNS A DATAFRAME
def page_scraper(html):
    soup=BeautifulSoup(html)
    #GETTING DATA
    product_names=soup.select('h1[class*="manhattan--titleText--"]')
    product_real_price=soup.select('div[class*="manhattan--price-sale--"]')
    product_original_price=soup.select('div[class*="manhattan--price-original--"]')
    product_sell_qty=soup.select('span[class*="manhattan--trade--"]')
    product_rate=soup.select('span[class*="manhattan--evaluation--"]')
    product_delivery_info=soup.select('div[class*="manhattan--topList--"]')
    product_official_store=soup.select('span[class*="cards--store--"]')
    #APPENDING DATA INTO LISTS
    product_names_list=[product.text for product in product_names]
    product_real_price_list=[product.text for product in product_real_price]
    product_original_price_list=[product.text for product in product_original_price]
    product_sell_qty_list=[product.text for product in product_sell_qty]
    product_rate_list=[product.text for product in product_rate]
    product_delivery_info_list=[product.text for product in product_delivery_info]
    product_official_store_list=[product.text for product in product_official_store]
    #CREATING DATAFRAME
    df=pd.DataFrame({'Product Name':product_names_list,
                            'Price':product_real_price_list,
                            'Original Price':product_original_price_list,
                            'Sell Quantity':product_sell_qty_list,
                            'Official Store':product_official_store_list,
                            'Rating':product_rate_list,
                            'delivery_info' : product_delivery_info_list
                            })
    return df

-----MAIN-----

In [11]:
#MAIN
driver = starting_setup()
url=driver.current_url
html=driver.page_source
global_df=pd.DataFrame()
print('WE ARE IN THE FIRST PAGE')
for i in range (2,50):
    driver.get(url)
    time_randomizer(2,7)
    
    for y in range (2,7):
        height = y*1080
        driver.execute_script(f"window.scrollTo(0, {height})") 
        time_randomizer(1,4)
    html=driver.page_source
    print('......Collecting data.......')
    df=page_scraper(html)
    global_df=pd.concat([global_df,df],ignore_index=True)
    if "page=" in url:
        url=url.replace('page=/d+',f'page={i}')
        print(f'WE ARE IN THE PAGE NUMBER {i}')
    else:
        url= f'{url}&page=2'
        print(url)
        print(f'WE ARE IN THE PAGE NUMBER {i}')
global_df.to_csv()
driver.quit()

Webpage Title: AliExpress - Compra online de Electrónica, Moda, Casa y jardín, Deportes y ocio, Motor y seguridad, y más. - AliExpress - AliExpress
Successful connection 

WE ARE IN THE FIRST PAGE
......Collecting data.......
https://es.aliexpress.com/category/204003301/video-games.html?CatId=204003301&g=y&isCategoryBrowse=true&isrefine=y&sortType=total_tranpro_desc&trafficChannel=main&isFavorite=y&page=2
WE ARE IN THE PAGE NUMBER 2
......Collecting data.......
WE ARE IN THE PAGE NUMBER 3
......Collecting data.......
WE ARE IN THE PAGE NUMBER 4
......Collecting data.......
WE ARE IN THE PAGE NUMBER 5
......Collecting data.......
WE ARE IN THE PAGE NUMBER 6
......Collecting data.......
WE ARE IN THE PAGE NUMBER 7
......Collecting data.......
WE ARE IN THE PAGE NUMBER 8
......Collecting data.......
WE ARE IN THE PAGE NUMBER 9
......Collecting data.......
WE ARE IN THE PAGE NUMBER 10
......Collecting data.......
WE ARE IN THE PAGE NUMBER 11
......Collecting data.......
WE ARE IN THE PAGE 

In [49]:
global_df.to_csv('global_df_not_cleaned')

In [21]:
global_df['Rating'] = global_df['Rating'].astype(float)

In [32]:
global_df['Price'] = global_df['Price'].str.replace('€','').str.strip()
global_df['Price'] = global_df['Price'].str.replace(',','.').astype(float)
global_df.head()

In [48]:
print(num_null)
global_df['Original Price'].str.replace(' ','')
num_null = global_df['Original Price'].isnull().sum()
global_df['Original Price'].astype(float)

0


ValueError: could not convert string to float: ''

In [43]:
#global_df['Original Price'] = global_df['Original Price'].str.replace('€','').str.strip()
global_df['Original Price'] = global_df['Original Price'].str.replace(' ','')
num_null = global_df['Original Price'].isnull().sum()
print("The Quantity of null values in the column 'Original Price':", num_null)
print("Null values filled as in column: 'Price'")
global_df['Original Price'].fillna(global_df['Price'], inplace=True)
num_null = global_df['Original Price'].isnull().sum()
print("The Quantity of null values in the column 'Original Price' now is:", num_null)
global_df['Original Price'] = global_df['Original Price'].str.replace(' ','')
global_df['Original Price'] = global_df['Original Price'].str.replace(',','.')
global_df['Original Price'] = global_df['Original Price'].astype(float)
global_df.head()

The Quantity of null values in the column 'Original Price': 0
Null values filled as in column: 'Price'
The Quantity of null values in the column 'Original Price' now is: 0


ValueError: could not convert string to float: ''

In [36]:
a = 

Product Name       object
Price             float64
Original Price     object
Sell Quantity      object
Official Store     object
Rating            float64
delivery_info      object
dtype: object

In [8]:
#PRUEBA PARA DATOS DE ENVIO

#SCRIPT PARA LLEGAR A LA PAGINA DE VIDEOJUEGOS CON TODO SELECCIONADO YA
driver = starting_setup()
url=driver.current_url
html=driver.page_source





Webpage Title: AliExpress - Compra online de Electrónica, Moda, Casa y jardín, Deportes y ocio, Motor y seguridad, y más. - AliExpress - AliExpress
Successful connection 



In [9]:
#INTENTO CAMBIANDO LA FUENTE DE LOS DATOS
'''ANTES OBTENIA LOS DATOS DE ENVIO CON EL SIGUIENTE TAG:
 product_shipping_info=soup.select('span[class*="tag--textStyle--"]')
 VAMOS A INTENTAR CAMBIARLO'''
soup=BeautifulSoup(html)
product_shipping_info=soup.select('div[class*="manhattan--topList--"]')
product_shipping_info_list=[product.text for product in product_shipping_info_1]
print(len(product_shipping_info_list))
product_shipping_info_list

59


['Envío gratis sobre 10€ · Entrega en 9 días',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 'Envío gratis · entrega en 9 días',
 'Envío gratis · entrega en 9 días',
 'Envío gratis',
 'Envío gratis',
 'Envío gratis',
 'Entrega en 10 por 12€Envío gratis',
 'Envío gratis',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 '+envío: 0,28€',
 'Entrega en 10 por 12€Envío gratis',
 '+envío: 0,25€',
 'Envío gratis · entrega en 9 días',
 'Envío gratis',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 'Entrega en 10 por 12€Envío gratis',
 'Entrega en 10 por 12€+envío: 0,34€',
 'Envío gratis · entrega en 9 días',
 'Envío gratis · entrega en 9 días',
 'Envío gratis',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 'Entrega en 10 por 12€Envío gratis',
 'Envío gratis sobre 10€ · Entrega en 9 días',
 'Entrega en 10 por 12€Envío gratis',
 'Entrega en 10 por 12€Envío gratis',
 'Entrega en 10 por 12€Envío gratis',
 'Envío gratis sobre 10€ · Entrega en 9 día

In [239]:
'''product_shipping_time=[]
product_free_shipping=[]
product_discount=[]
for text in product_shipping_info_list:
    if 'dto'or'descuento' in text:
        product_discount.append(text)
    else:
        product_discount.append('-')
    if 'Envío' in text:
        product_free_shipping.append(text)
    else:
        product_free_shipping.append('-')
    if 'Entrega' in text:
        product_shipping_time.append(text)
    else:
        product_shipping_time.append('-')

print(len(product_shipping_time))
print(len(product_free_shipping))
print(len(product_discount))
'''

115
115
115


In [ ]:
if 'Envío gratis' in global_df['delivery_info']:
    